In [1]:
import json

with open(
    "F:\\automate-accounts\\automate-accouts-software-\\spacy_training_data.json",
    "r",
    encoding="utf-8",
) as f:
    spacy_data = json.load(f)

# Example: First entry
print(spacy_data[0:50])

[["\n--- Page 1 ---\nTHE VENETIAN* THE PALAZZO\nLAS VEGAS\n3355 Las Vegas Blvd. So.\nLas Vegas, NV 89109\nDATE REFERENCE NO. DESCRIPTION CHARGES PAYMENTS/ BALANCE\nCREDITS!)\n11/25/18 434289101289 RESORT FEE 22.68\nRESORT FEE-$20 PIUS TAX\n11/25/18 434289101290 ROOM CHARGE UPGRADE VEN 51.02\nUPGRADE CHARGE - ADDITION\n11/25/18 434289113392 ROOM CHARGE VE33310 209.00\nTAC2 27.96\n11/25/18 434280912999 APPLIED DEPOSIT 236.96\n11/26/18 434299101737 RESORT FEE 22.68\nRESORT FEE-$20 PLUS TAX\n11/26/18 434299101738 ROOM CHARGE UPGRADE VEN 51.02\nUPGRADE CHARGE - ADDITION\n11/26/18 434299114276 ROOM CHARGE VE33310 209.00\nTAC2 27.96\n11/27/18 434309101779 RESORTFFF 22.68\nRESORT FEE-$20 PLUS TAX\n11/27/18 434309101780 ROOM CHARGE UPGRADE VEN 51.02\nUPGRADE CHARGE - ADDITION\n11/27/18 434309114682 ROOM CHARGE VE33310 209.00\nTAC2 27.96\n11/28/18 434319101739 RESORT FEE 22.68\nRESORT FEE - $20 PLUS TAX\n11/28/18 434319101740 ROOM CHARGE UPGRADE VEN 51.02\nUPGRADE CHARGE - ADDITION\n11/28/18 434

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def spacy_to_bio(text, entities, tokenizer):
    """
    Convert SpaCy JSON (start, end, label) annotations into BIO tags for HuggingFace.
    """
    # Tokenize with offsets
    tokens = tokenizer(text, return_offsets_mapping=True, truncation=True)
    offset_mapping = tokens["offset_mapping"]

    labels = ["O"] * len(offset_mapping)

    for start, end, label in entities:
        for i, (tok_start, tok_end) in enumerate(offset_mapping):
            if tok_start >= start and tok_end <= end:
                if labels[i] == "O":
                    labels[i] = "B-" + label
                elif labels[i].startswith("B-") or labels[i].startswith("I-"):
                    labels[i] = "I-" + label

    return tokens["input_ids"], tokens["attention_mask"], labels

In [ ]:
all_input_ids, all_attention_masks, all_labels = [], [], []

for text, ann in spacy_data:
    entities = ann["entities"]
    input_ids, attention_mask, labels = spacy_to_bio(text, entities, tokenizer)

    all_input_ids.append(input_ids)
    all_attention_masks.append(attention_mask)
    all_labels.append(labels)

In [ ]:
from datasets import Dataset

dataset = Dataset.from_dict(
    {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_masks,
        "labels": all_labels,
    }
)

print(dataset[0:10])

{'input_ids': [[101, 118, 118, 118, 8500, 122, 118, 118, 118, 7462, 159, 11680, 11943, 9984, 2249, 115, 7462, 8544, 10783, 5301, 5301, 2346, 10722, 1708, 159, 17020, 10719, 27461, 1571, 5976, 6554, 139, 1233, 1964, 1181, 119, 1573, 119, 5976, 6554, 117, 151, 2559, 5840, 10424, 1580, 141, 13821, 2036, 155, 14663, 9637, 11680, 10954, 24819, 119, 18581, 10844, 20595, 2101, 21669, 11414, 24890, 12426, 16523, 1708, 8544, 3663, 14424, 15681, 1708, 120, 12465, 10783, 15517, 2036, 15531, 10069, 12150, 1708, 106, 114, 1429, 120, 1512, 120, 1407, 3887, 23117, 1604, 1580, 10424, 11964, 1604, 1580, 155, 9919, 9565, 1942, 143, 27073, 1659, 119, 5599, 155, 9919, 9565, 1942, 143, 27073, 118, 109, 1406, 153, 2240, 13329, 157, 1592, 3190, 1429, 120, 1512, 120, 1407, 3887, 23117, 1604, 1580, 10424, 11964, 21500, 155, 2346, 13041, 24890, 12426, 16523, 19753, 27617, 14569, 2036, 159, 11680, 4062, 119, 5507, 19753, 27617, 14569, 2036, 24890, 12426, 16523, 118, 5844, 17243, 21669, 11414, 1429, 120, 1512, 12

In [9]:
unique_labels = sorted({lab for seq in all_labels for lab in seq})
label2id = {lab: i for i, lab in enumerate(unique_labels)}
id2label = {i: lab for lab, i in label2id.items()}

label2id

{'B-ADDRESS': 0,
 'B-BUSINESS_TYPE': 1,
 'B-CARD_DETAILS': 2,
 'B-CUSTOMER_EMAIL': 3,
 'B-ITEMS': 4,
 'B-MERCHANT_EMAIL_ID': 5,
 'B-MERCHANT_NAME': 6,
 'B-MERCHANT_PHONE': 7,
 'B-PAYMENT_METHOD': 8,
 'B-PURCHASED_AT': 9,
 'B-PURCHASE_QUANTITY': 10,
 'B-TOTAL_AMOUNT': 11,
 'B-TRANSACTION_ID': 12,
 'I-BUSINESS_TYPE': 13,
 'I-CARD_DETAILS': 14,
 'I-CUSTOMER_EMAIL': 15,
 'I-ITEMS': 16,
 'I-MERCHANT_EMAIL_ID': 17,
 'I-MERCHANT_PHONE': 18,
 'I-TRANSACTION_ID': 19,
 'O': 20}